In [1]:
import scripts.IO_utils as io
import scripts.NN as NN

In [2]:
import os
import sys
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import random
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def auc_roc(e, p):
    fpr, tpr, thresh = metrics.roc_curve(e, p, pos_label=1)
    AUC = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("AUC = " + str(AUC))

In [4]:
pos_seqs = io.read_positives()
neg_seqs = io.read_negatives(pos_seqs)
pos_seqs = io.to_binary(pos_seqs)
neg_seqs = io.to_binary(neg_seqs)

/Users/egilbertson/anaconda3/lib/python3.7/site-packages/Bio/Seq.py:182: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  BiopythonWarning,


In [5]:
pos, neg = io.downsample_and_label(pos_seqs, neg_seqs, 100)

In [6]:
train, test = io.split_training_testing(pos, neg, 0.8)
random.shuffle(train)
random.shuffle(test)

In [7]:
test = io.string_to_array(test)
train = io.string_to_array(train)

In [ ]:
nn = NN.NeuralNetwork(input = train, test_set = test, output_dim = 1, reg_term = 0, rounds=100)
err = nn.fit()


In [ ]:
mse_train = np.average(np.square(nn.backprop()))
mse_train

In [ ]:
p, mse_test = nn.predict()

In [ ]:
mse_test

In [ ]:
test_e = nn.test_exp

In [ ]:
auc_roc(test_e, p)

In [ ]:
io.cross_validation(pos, neg, k=5, reg_term= 0, rounds=100, learn_rate=0.01, num_hidden=3)

In [ ]:
io.cross_validation(pos, neg, k=5, reg_term= 0, rounds=1000, learn_rate=0.01, num_hidden=3)

In [8]:
test_seqs_string = io.read_test()

In [9]:
len(test_seqs_string)

3195

In [10]:
test_seqs = io.to_binary(test_seqs_string)

In [11]:
test_seqs = io.string_to_array(test_seqs)

In [12]:
nn = NN.NeuralNetwork(input = train, test_set = test_seqs, output_dim = 1, reg_term = 0, rounds=10000, expected_out = 'blind')
err = nn.fit()

In [23]:
p = nn.predict()

In [24]:
df = pd.DataFrame()

In [25]:
df = df.append(test_seqs_string)
df['Prediction'] = ''

In [26]:
p = p[0]

In [27]:
for i in range(len(p)):
    df.Prediction[i] = p[i][0]

In [28]:
df

,0,Prediction
0,AAAAAAACGCAACTAAT,4.82356e-12
1,AAAAACACACATCTGGC,4.82356e-12
2,AAAACCAAACACCTGAA,0.731059
3,AAAACCAAACAGCTCAG,0.731059
4,AAAACCAAACATATCAA,0.731059
...,...,...
3190,TTGTTCATACATTCTTA,4.82356e-12
3191,TTGTTCATATCTTCCAA,4.82356e-12
3192,TTTCTCATACACCTTTA,4.82356e-12
3193,TTTTCCAAGCATTTGTA,4.82356e-12


In [29]:
df.to_csv('attempt1.tsv', sep ='\t', header=False, index=False)